## 데이터 불러오기

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import requests
import matplotlib.pyplot as plt
import pubchempy as pcp

In [3]:
subacute_up_gene_np = pd.read_csv("/data1/project/eunyi/tbi/0.subacute_up_gene_np.csv")
subacute_down_gene_np = pd.read_csv("/data1/project/eunyi/tbi/0.subacute_down_gene_np.csv")
acute_up_gene_np = pd.read_csv("/data1/project/eunyi/tbi/0.acute_up_gene_np.csv")
acute_down_gene_np = pd.read_csv("/data1/project/eunyi/tbi/0.acute_down_gene_np.csv")

In [4]:
subacute_up_cid_np = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/1.subacute_up_cid_np")
subacute_up_cid_np

,gene,np_score,CID
0,Ephx1,0.010654,95783
1,Ephx1,0.010654,86766827
2,Ephx1,0.010654,7547
3,Ephx1,0.010654,238899
4,Ephx1,0.010654,45269948
...,...,...,...
629456,Pde7b,0.000000,101886684
629457,Pde7b,0.000000,101886685
629458,Pde7b,0.000000,50993785
629459,Pde7b,0.000000,50993786


In [5]:
subacute_down_cid_np = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/1.subacute_down_cid_np")
subacute_down_cid_np

,gene,np_score,CID
0,Cd44,0.010248,90655421
1,Cd44,0.010248,90655419
2,Cd44,0.010248,16217499
3,Cd44,0.010248,73441667
4,Cd44,0.010248,90655422
...,...,...,...
629456,Dhrs9,0.000000,2758833
629457,Dhrs9,0.000000,10330671
629458,Dhrs9,0.000000,2758835
629459,Dhrs9,0.000000,44434500


In [6]:
acute_up_cid_np = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/1.acute_up_cid_np")
acute_up_cid_np

,gene,np_score,CID
0,Xdh,0.010584,6477682
1,Xdh,0.010584,6477683
2,Xdh,0.010584,6477684
3,Xdh,0.010584,6477685
4,Xdh,0.010584,2094
...,...,...,...
629456,Proc,0.000000,20595260
629457,Proc,0.000000,44274017
629458,Proc,0.000000,10095865
629459,Proc,0.000000,9911502


In [7]:
acute_down_cid_np = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/1.acute_down_cid_np")
acute_down_cid_np

,gene,np_score,CID
0,Kit,0.010802,57496699
1,Kit,0.010802,118697071
2,Kit,0.010802,46182735
3,Kit,0.010802,118698116
4,Kit,0.010802,118697041
...,...,...,...
629456,Abhd12,0.000000,3034010
629457,Abhd12,0.000000,3034010
629458,Abhd12,0.000000,137651267
629459,Abhd16a,0.000000,5063


In [7]:
# 압축 해제된 파일 불러오기 (예: CID-Synonym-filtered 파일)
synonym_data = pd.read_csv('CID-Synonym-filtered', sep='\t', header=None, names=['CID', 'Synonym'])

# CID와 Drug Name(Synonym) 매핑
print(synonym_data.head())

   CID               Synonym
0    1   Acetyl-DL-carnitine
1    1       acetylcarnitine
2    1    DL-Acetylcarnitine
3    1  DL-O-Acetylcarnitine
4    1            14992-62-2


In [17]:
cid_title_data = pd.read_csv('/data1/project/eunyi/tbi/drug_gene/CID-Title', sep='\t', header=None, names=['CID', 'Drug Name'], encoding='ISO-8859-1')

# 결과 확인
print(cid_title_data.head())

   CID                                          Drug Name
0    1                                Acetyl-DL-carnitine
1    2    [2-(Acetyloxy)-3-carboxypropyl]trimethylazanium
2    3  5,6-Dihydroxycyclohexa-1,3-diene-1-carboxylic ...
3    4                                 1-Aminopropan-2-ol
4    5                      3-Amino-2-oxopropyl phosphate


In [9]:
# CID의 유니크한 개수 계산
unique_cid_count = synonym_data['CID'].nunique()

# 결과 출력
print(f"CID의 유니크한 개수: {unique_cid_count}")


CID의 유니크한 개수: 56345079


In [16]:
# Synonym 데이터 불러오기
synonym_data = pd.read_csv('CID-Synonym-filtered', sep='\t', header=None, names=['CID', 'Synonym'])

# 1. 숫자가 포함된 Synonym (CAS 번호 등) 제외
filtered_data = synonym_data[~synonym_data['Synonym'].str.match(r'\d{2,7}-\d{2}-\d')]

# 2. 화학적 명칭 또는 복잡한 구조의 이름을 제외 (하이픈, 괄호 등이 포함된 이름 필터링)
filtered_data = filtered_data[~filtered_data['Synonym'].str.contains(r'(\-|\(|\))')]

# 3. 각 CID에 대해 첫 번째로 간결한 이름 선택 (중복 제거)
first_drug_name_per_cid = filtered_data.drop_duplicates(subset=['CID'], keep='first')
first_drug_name_per_cid

/tmp/ipykernel_337001/1337392541.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_data = filtered_data[~filtered_data['Synonym'].str.contains(r'(\-|\(|\))')]


,CID,Synonym
1,1,acetylcarnitine
44,2,Levocarnitine acetyl hydrochloride
57,3,SCHEMBL638361
65,4,Isopropanolamine
179,5,CHEBI:1449
...,...,...
101898294,171905550,NSC831558
101898296,171905551,NSC831547
101898298,171905552,NSC831479
101898300,171905553,LSB tartrate


In [10]:
# CID별로 첫 번째로 나오는 Synonym만 선택 (중복 제거)
first_synonym_per_cid = synonym_data.drop_duplicates(subset=['CID'], keep='first')
first_synonym_per_cid

,CID,Synonym
0,1,Acetyl-DL-carnitine
41,2,[2-(ACETYLOXY)-3-CARBOXYPROPYL]TRIMETHYLAZANIUM
53,3,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
62,4,1-Aminopropan-2-ol
173,5,3-Amino-2-oxopropyl phosphate
...,...,...
101898298,171905552,NSC831479
101898300,171905553,LSB tartrate
101898304,171905554,N-2-pentyl lysergamide tartrate
101898308,171905555,N-3-pentyl lysergamide tartrate


In [15]:
# 1. 숫자 형식의 CAS 번호(예: 12345-67-8) 패턴을 포함하는 Synonym 제외
filtered_data = synonym_data[~synonym_data['Synonym'].str.match(r'\d{2,7}-\d{2}-\d')]

# 2. CID별로 첫 번째로 나오는 Synonym 선택 (중복 제거)
first_synonym_per_cid = filtered_data.drop_duplicates(subset=['CID'], keep='first')

first_synonym_per_cid

,CID,Synonym
0,1,Acetyl-DL-carnitine
41,2,[2-(ACETYLOXY)-3-CARBOXYPROPYL]TRIMETHYLAZANIUM
53,3,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
62,4,1-Aminopropan-2-ol
173,5,3-Amino-2-oxopropyl phosphate
...,...,...
101898298,171905552,NSC831479
101898300,171905553,LSB tartrate
101898304,171905554,N-2-pentyl lysergamide tartrate
101898308,171905555,N-3-pentyl lysergamide tartrate


In [11]:
first_synonym_per_cid.to_csv("/data1/project/eunyi/tbi/drug_gene/cid_to_drug_info.csv", index=False)

In [14]:
cid_to_drug_info = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/cid_to_drug_info.csv")
cid_to_drug_info

,CID,Synonym
0,1,Acetyl-DL-carnitine
1,2,[2-(ACETYLOXY)-3-CARBOXYPROPYL]TRIMETHYLAZANIUM
2,3,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
3,4,1-Aminopropan-2-ol
4,5,3-Amino-2-oxopropyl phosphate
...,...,...
56345074,171905552,NSC831479
56345075,171905553,LSB tartrate
56345076,171905554,N-2-pentyl lysergamide tartrate
56345077,171905555,N-3-pentyl lysergamide tartrate


In [8]:
# 1. 숫자가 포함된 Synonym (CAS 번호 등) 제외
filtered_data = synonym_data[~synonym_data['Synonym'].str.contains(r'\d')]

# 2. 화학적 명칭이나 구조적 명칭 필터링 (IUPAC 명칭 등 제외)
# 기본적으로 매우 긴 화학적 명칭이나 복잡한 화합물은 제외 (패턴을 이용해 필터링 가능)
filtered_data = filtered_data[~filtered_data['Synonym'].str.contains(r'(\-|\(|\))')]

# 3. 각 CID에 대해 첫 번째로 간결한 이름 선택
first_synonym_per_cid = filtered_data.drop_duplicates(subset=['CID'], keep='first')
first_synonym_per_cid

/tmp/ipykernel_337001/2532819244.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_data = filtered_data[~filtered_data['Synonym'].str.contains(r'(\-|\(|\))')]


,CID,Synonym
1,1,acetylcarnitine
44,2,Levocarnitine acetyl hydrochloride
65,4,Isopropanolamine
187,6,Dinitrochlorobenzene
356,11,Ethylene dichloride
...,...,...
101897981,171905321,sulfopantetheine
101897985,171905325,tridecyl alcohol propoxy sulfate
101898091,171905431,Titanium picolinate
101898098,171905438,distearoylphosphaethanolamine


## CID-Title 데이터로 진행

In [9]:
cid_title_data = pd.read_csv('/data1/project/eunyi/tbi/drug_gene/CID-Title', sep='\t', header=None, names=['CID', 'Name'], encoding='ISO-8859-1')
cid_title_data.head(10)

,CID,Name
0,1,Acetyl-DL-carnitine
1,2,[2-(Acetyloxy)-3-carboxypropyl]trimethylazanium
2,3,"5,6-Dihydroxycyclohexa-1,3-diene-1-carboxylic ..."
3,4,1-Aminopropan-2-ol
4,5,3-Amino-2-oxopropyl phosphate
5,6,"1-Chloro-2,4-dinitrobenzene"
6,7,9-Ethyladenine
7,8,"2,3-Dihydroxy-3-methylpentanoic acid"
8,9,"(2,3,4,5,6-Pentahydroxycyclohexyl) dihydrogen ..."
9,11,"1,2-Dichloroethane"


In [11]:
merged_df = subacute_up_cid_np.merge(cid_title_data, on='CID', how='left')
merged_df

,gene,np_score,CID,Name
0,Ephx1,0.010654,95783,"1,3-Bis(3,4-dichlorophenyl)urea"
1,Ephx1,0.010654,86766827,"US8815951, Epoxide Hydrolase Inhibitor 11"
2,Ephx1,0.010654,7547,Triclocarban
3,Ephx1,0.010654,238899,1-(4-Chlorophenyl)-3-cyclohexylurea
4,Ephx1,0.010654,45269948,4-(3-hydroxy-4-methylisoxazol-5-yl)-N-(2-pheny...
...,...,...,...,...
629456,Pde7b,0.000000,101886684,1-(2-Bromophenyl)sulfanyl-4-chloro-2-nitrobenzene
629457,Pde7b,0.000000,101886685,2-(2-Bromophenyl)sulfanyl-4-chloro-1-nitrobenzene
629458,Pde7b,0.000000,50993785,5-(1h-Benzimidazol-2-Ylmethylsulfanyl)-2-Methy...
629459,Pde7b,0.000000,50993786,"2,9-Dimethyl-5-(2-(1-methyl-4-phenyl-1H-imidaz..."


In [20]:
# Name별로 gene 수(cnt_gene)와 np_score의 평균(mean_np_score) 계산
grouped_df = merged_df.groupby('Name').agg(
    cnt_gene=('gene', 'nunique'),  # 고유한 gene 수 계산
    mean_npscore=('np_score', 'mean')  # np_score 평균 계산
).reset_index()

# mean_np_score 소수점 6자리로 반올림
grouped_df['mean_npscore'] = grouped_df['mean_npscore'].round(6)

# mean_np_score 내림차순으로 정렬
grouped_df = grouped_df.sort_values(by='mean_npscore', ascending=False)

# 결과 확인
grouped_df.head(10)

,Name,cnt_gene,mean_npscore
59542,1-Cyclohexyl-3-(pyridin-4-ylmethyl)urea,1,0.010654
59546,1-Cyclohexyl-3-[(4-nitrophenyl)methyl]urea,1,0.010654
19540,(3-phenylphenyl) N-(pyridin-3-ylmethyl)carbamate,1,0.010654
19541,(3-phenylphenyl) N-(pyridin-4-ylmethyl)carbamate,1,0.010654
65888,1-[2-(4-Chlorophenyl)ethyl]-3-cyclohexylurea,1,0.010654
59573,1-Cyclohexyl-3-cyclopentylurea,1,0.010654
59574,1-Cyclohexyl-3-dodecylurea,1,0.010654
59575,1-Cyclohexyl-3-hexyl urea,1,0.010654
59576,1-Cyclohexyl-3-isobutylurea,1,0.010654
59578,1-Cyclohexyl-3-naphthalen-1-ylurea,1,0.010654


In [21]:
grouped_df.to_csv('/data1/project/eunyi/tbi/drug_gene/1.subacute_up_name.csv', index=False)

In [22]:
subacute_up_name = pd.read_csv('/data1/project/eunyi/tbi/drug_gene/1.subacute_up_name.csv')
subacute_up_name

,Name,cnt_gene,mean_npscore
0,1-Cyclohexyl-3-(pyridin-4-ylmethyl)urea,1,0.010654
1,1-Cyclohexyl-3-[(4-nitrophenyl)methyl]urea,1,0.010654
2,(3-phenylphenyl) N-(pyridin-3-ylmethyl)carbamate,1,0.010654
3,(3-phenylphenyl) N-(pyridin-4-ylmethyl)carbamate,1,0.010654
4,1-[2-(4-Chlorophenyl)ethyl]-3-cyclohexylurea,1,0.010654
...,...,...,...
340315,3-Tert-butyl 5-methyl 4-(2-chloro-4-fluorophen...,1,0.000000
340316,"3-Tert-butyl 5-methyl 2,6-dimethyl-4-p-tolyl-1...",1,0.000000
340317,"3-Tert-butyl 5-methyl 2,6-dimethyl-4-o-tolyl-1...",1,0.000000
340318,"3-Tert-butyl 5-methyl 2,6-dimethyl-4-m-tolyl-1...",1,0.000000


In [23]:
merged_df = subacute_down_cid_np.merge(cid_title_data, on='CID', how='left')
merged_df

,gene,np_score,CID,Name
0,Cd44,0.010248,90655421,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...
1,Cd44,0.010248,90655419,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)b-GlcA
2,Cd44,0.010248,16217499,"1,2,3,4-Tetrahydroisoquinolin-5-amine"
3,Cd44,0.010248,73441667,"2-[(4-Methyl-1h-Imidazol-5-Yl)methyl]-1,2,3,4-..."
4,Cd44,0.010248,90655422,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...
...,...,...,...,...
629456,Dhrs9,0.000000,2758833,"2,3-Dihydro-1,4-benzodioxine-6-carboxylic acid"
629457,Dhrs9,0.000000,10330671,"(S)-2-(2,3-dihydrobenzo[b][1,4]dioxin-6-yl)pro..."
629458,Dhrs9,0.000000,2758835,"4-(2,3-Dihydro-1,4-benzodioxin-6-yl)-4-oxobuta..."
629459,Dhrs9,0.000000,44434500,2-(N-(2-fluorophenyl)pyrrol-2-yl) acetic acid


In [24]:
# Name별로 gene 수(cnt_gene)와 np_score의 평균(mean_np_score) 계산
grouped_df = merged_df.groupby('Name').agg(
    cnt_gene=('gene', 'nunique'),  # 고유한 gene 수 계산
    mean_npscore=('np_score', 'mean')  # np_score 평균 계산
).reset_index()

# mean_np_score 소수점 6자리로 반올림
grouped_df['mean_npscore'] = grouped_df['mean_npscore'].round(6)

# mean_np_score 내림차순으로 정렬
grouped_df = grouped_df.sort_values(by='mean_npscore', ascending=False)

# 결과 확인
grouped_df.head(10)

,Name,cnt_gene,mean_npscore
219521,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)b-GlcA,1,0.010248
219520,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
219519,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
219518,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
219517,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
48200,"1,2,3,4-Tetrahydroisoquinolin-5-amine",1,0.010248
99412,"2-[(4-Methyl-1h-Imidazol-5-Yl)methyl]-1,2,3,4-...",1,0.010248
88264,2-(4-Chlorobenzyl)-3-(cyclohexanecarbonyloxy)-...,1,0.010075
88265,"2-(4-Chlorobenzyl)-3-(isobutoxycarbonyloxy)-7,...",1,0.010075
187393,"5-{(2S,3R,4S)-3,4-Dihydroxy-2-[(1R,2R)-2-((2R,...",1,0.010075


In [25]:
grouped_df.to_csv('/data1/project/eunyi/tbi/drug_gene/1.subacute_down_name.csv', index=False)

In [26]:
subacute_down_name = pd.read_csv("/data1/project/eunyi/tbi/drug_gene/1.subacute_down_name.csv")
subacute_down_name

,Name,cnt_gene,mean_npscore
0,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)b-GlcA,1,0.010248
1,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
2,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
3,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
4,GlcNAc(b1-4)GlcA(b1-3)GlcNAc(b1-4)GlcA(b1-3)Gl...,1,0.010248
...,...,...,...
340315,"3-Hydroxy-benzo[4,5]thieno[3,2-b]furan-2-carbo...",1,0.000000
340316,3-Hydroxy-benzaldehyde4-(beta-D-glucopyranosyl...,1,0.000000
340317,3-Hydroxy-N-((S)-2-oxotetrahydrofuran-3-yl)dod...,1,0.000000
340318,"3-Hydroxy-9H-dipyrido[2,3-b;4',3'-d]pyrrole-6-...",1,0.000000


In [27]:
merged_df = acute_up_cid_np.merge(cid_title_data, on='CID', how='left')
merged_df

,gene,np_score,CID,Name
0,Xdh,0.010584,6477682,"5,7-Dihydroxy-2-(4-hydroxystyryl)-4H-chromen-4..."
1,Xdh,0.010584,6477683,"2-Styrylchromone derivate, 3a"
2,Xdh,0.010584,6477684,"2-Styrylchromone derivate, 3b"
3,Xdh,0.010584,6477685,"2-Styrylchromone derivate, 3c"
4,Xdh,0.010584,2094,NaN
...,...,...,...,...
629456,Proc,0.000000,20595260,2-(3-Methanesulfonyl-naphthalen-2-yl)-5-methyl...
629457,Proc,0.000000,44274017,"1H-Pyrazole-5-carboxamide, 1-(6-chloro-2-napht..."
629458,Proc,0.000000,10095865,(2r)-2-(5-Chloro-2-Thienyl)-N-{(3s)-1-[(1s)-1-...
629459,Proc,0.000000,9911502,2-(5-Chloro-2-Thienyl)-N-{(3s)-1-[(1s)-1-Methy...


In [28]:
# Name별로 gene 수(cnt_gene)와 np_score의 평균(mean_np_score) 계산
grouped_df = merged_df.groupby('Name').agg(
    cnt_gene=('gene', 'nunique'),  # 고유한 gene 수 계산
    mean_npscore=('np_score', 'mean')  # np_score 평균 계산
).reset_index()

# mean_np_score 소수점 6자리로 반올림
grouped_df['mean_npscore'] = grouped_df['mean_npscore'].round(6)

# mean_np_score 내림차순으로 정렬
grouped_df = grouped_df.sort_values(by='mean_npscore', ascending=False)

# 결과 확인
grouped_df.head(10)

,Name,cnt_gene,mean_npscore
100522,2-[(Z)-[6-(carboxymethoxy)-3-oxo-1-benzofuran-...,1,0.010584
84552,"2-(2-Methylbenzylamino)-4-methyl-1,3-thiazole-...",1,0.010584
84559,2-(2-Methylphenyl)-1-[(2-methylphenyl)methyl]b...,1,0.010584
10186,"(2R,3R)-2-(3,4-dihydroxyphenyl)-3-hydroxy-5,6,...",1,0.010584
84997,2-(2-hydroxyethoxy)-5-(5-(2-methylpyridin-4-yl...,1,0.010584
170497,4-{3-Cyano-4-[2-(2-methoxyethoxy)phenyl]pyrrol...,1,0.010584
85052,2-(2-methoxyethoxy)-5-(5-(2-methylpyridin-4-yl...,1,0.010584
159752,4-[3-(2-Chlorophenyl)-4-cyanopyrrol-1-yl]-2-hy...,1,0.010584
203859,"7-methyl-2-((4-nitrophenoxy)methyl)-5H-[1,3,4]...",1,0.010584
203865,"7-methyl-2-(m-tolyloxymethyl)-5H-[1,3,4]thiadi...",1,0.010584


In [30]:
grouped_df.to_csv('/data1/project/eunyi/tbi/drug_gene/1.acute_up_name.csv', index=False)

In [31]:
acute_up_name = pd.read_csv('/data1/project/eunyi/tbi/drug_gene/1.acute_up_name.csv')
acute_up_name

,Name,cnt_gene,mean_npscore
0,2-[(Z)-[6-(carboxymethoxy)-3-oxo-1-benzofuran-...,1,0.010584
1,"2-(2-Methylbenzylamino)-4-methyl-1,3-thiazole-...",1,0.010584
2,2-(2-Methylphenyl)-1-[(2-methylphenyl)methyl]b...,1,0.010584
3,"(2R,3R)-2-(3,4-dihydroxyphenyl)-3-hydroxy-5,6,...",1,0.010584
4,2-(2-hydroxyethoxy)-5-(5-(2-methylpyridin-4-yl...,1,0.010584
...,...,...,...
340315,5-[4-(Dimethylamino)phenyl]-6-[(6-Morpholin-4-...,1,0.000000
340316,"5-[4-Amino-1-(2,2-diethoxyethyl)pyrazolo[3,4-d...",1,0.000000
340317,5-[4-Amino-1-[(2-morpholin-4-ylquinolin-3-yl)m...,4,0.000000
340318,5-[4-Amino-1-[(2-phenylquinolin-3-yl)methyl]py...,4,0.000000


In [32]:
merged_df = acute_down_cid_np.merge(cid_title_data, on='CID', how='left')
merged_df

,gene,np_score,CID,Name
0,Kit,0.010802,57496699,N-[2-(4-{[3-chloro-4-(3-chlorophenoxy)phenyl]a...
1,Kit,0.010802,118697071,tert-butyl 4-[4-[3-[[(1S)-1-(4-chlorophenyl)-3...
2,Kit,0.010802,46182735,"(Z)-1-[2-Oxo-3-(1H-pyrrol-2-ylmethylene)-2,3-d..."
3,Kit,0.010802,118698116,methyl 4-[4-[3-[(4-chloro-2-fluorophenyl)methy...
4,Kit,0.010802,118697041,N-[(1S)-1-(4-chlorophenyl)propyl]-1-[6-[1-(2-h...
...,...,...,...,...
629456,Abhd12,0.000000,3034010,Orlistat
629457,Abhd12,0.000000,3034010,Orlistat
629458,Abhd12,0.000000,137651267,(4-Benzylpiperidin-1-yl)-[5-(4-hydroxyphenyl)-...
629459,Abhd16a,0.000000,5063,"1,6-Bis(cyclohexyloximinocarbonyl)hexane"


In [33]:
# Name별로 gene 수(cnt_gene)와 np_score의 평균(mean_np_score) 계산
grouped_df = merged_df.groupby('Name').agg(
    cnt_gene=('gene', 'nunique'),  # 고유한 gene 수 계산
    mean_npscore=('np_score', 'mean')  # np_score 평균 계산
).reset_index()

# mean_np_score 소수점 6자리로 반올림
grouped_df['mean_npscore'] = grouped_df['mean_npscore'].round(6)

# mean_np_score 내림차순으로 정렬
grouped_df = grouped_df.sort_values(by='mean_npscore', ascending=False)

# 결과 확인
grouped_df.head(10)

,Name,cnt_gene,mean_npscore
72341,"1-[6-(1-cyclobutylpyrazol-4-yl)pyrrolo[2,1-f][...",1,0.010802
72671,1-[6-[4-(2-hydroxypropan-2-yl)phenyl]pyrrolo[2...,1,0.010802
244818,N-[(1S)-1-(4-chlorophenyl)propyl]-1-[6-[1-[(2R...,1,0.010802
336995,propan-2-yl 4-[4-[3-[(4-methylphenyl)methylcar...,1,0.010802
231643,N-(2-aminopyrimidin-5-yl)-6-chloro-3-(3-(trifl...,1,0.010802
72670,1-[6-[4-(2-hydroxypropan-2-yl)phenyl]pyrrolo[2...,1,0.010802
203696,7-fluoro-N-[5-[5-[3-hydroxy-3-(trifluoromethyl...,1,0.010802
244819,N-[(1S)-1-(4-chlorophenyl)propyl]-1-[6-[1-[(2R...,1,0.010802
240060,N-(5-(5-(3-methoxy-3-(trifluoromethyl)cyclobut...,1,0.010802
250677,N-[(S)-(4-chlorophenyl)-cyclopropylmethyl]-1-[...,1,0.010802


In [34]:
grouped_df.to_csv('/data1/project/eunyi/tbi/drug_gene/1.acute_down_name.csv', index=False)

In [35]:
acute_down_name = pd.read_csv('/data1/project/eunyi/tbi/drug_gene/1.acute_down_name.csv')
acute_down_name

,Name,cnt_gene,mean_npscore
0,"1-[6-(1-cyclobutylpyrazol-4-yl)pyrrolo[2,1-f][...",1,0.010802
1,1-[6-[4-(2-hydroxypropan-2-yl)phenyl]pyrrolo[2...,1,0.010802
2,N-[(1S)-1-(4-chlorophenyl)propyl]-1-[6-[1-[(2R...,1,0.010802
3,propan-2-yl 4-[4-[3-[(4-methylphenyl)methylcar...,1,0.010802
4,N-(2-aminopyrimidin-5-yl)-6-chloro-3-(3-(trifl...,1,0.010802
...,...,...,...
340315,3-[4-Chloro-3-[7-methyl-12-(trifluoromethyl)-2...,2,0.000000
340316,3-[4-Chloro-3-(trifluoromethyl)phenyl]-1-[1-[3...,1,0.000000
340317,"3-[4-Chloro-3-(trifluoromethyl)phenyl]-1-(2,6-...",1,0.000000
340318,"3-[4-Chloro-3-(trifluoromethoxy)phenyl]-1-(2,6...",1,0.000000
